## Predictive Keyboard

### Loading Libraries:

In [15]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

### Loading the data

In [2]:
path = 'data.txt'
#text = open("data.txt", "r").read().lower()
text = open(path, encoding = 'utf8').read().lower()
print('corpus length:', len(text))

corpus length: 581886


### Preprocessing:

In [3]:
# Tokenizing the words
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [27]:
words[0:0+5] # First five words

['project', 'gutenberg', 's', 'the', 'adventures']

In [28]:
words[1:1+5] # Next five words

['gutenberg', 's', 'the', 'adventures', 'of']

In [22]:
# Determining the Unique Words
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))


### Feature Engineering:

In [5]:
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['project', 'gutenberg', 's', 'the', 'adventures']
of


In [33]:
len(unique_words)

8201

### Creating X and Y variables:

In [6]:
# Assiging the prev_words to X and next_word to Y:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [7]:
print(X[0][0])

[False False False ... False False False]


### Building the model:

In [44]:
print(WORD_LENGTH, len(unique_words))

5 8201


In [8]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

### Training:

In [9]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history


Train on 103759 samples, validate on 5462 samples
Epoch 1/2
103759/103759 [==============================] - 203s 2ms/step - loss: 6.0189 - accuracy: 0.1077 - val_loss: 7.0812 - val_accuracy: 0.0998
Epoch 2/2
103759/103759 [==============================] - 188s 2ms/step - loss: 5.7773 - accuracy: 0.1459 - val_loss: 8.0806 - val_accuracy: 0.1053


### Saving the model:

In [10]:
model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))


### Loading the model:

In [16]:
model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

### Prediction:

In [11]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x
prepare_input("It is not a lack".lower())

it
is
not
a
lack


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

### Evaluation:

In [12]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [13]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

In [35]:
q =  "Your life will never be the same again"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[1:6])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 6))

correct sentence:  Your life will never be the same again
Sequence:  life will never be the
life
will
never
be
the
next possible words:  ['same', 'very', 'matter', 'police', 'little', 'name']
